# Build `MESH` Model Setup for the Bow River at Calgary

We use the `MESHFlow` Python package to build a `MESH` model setup for the Bow River at Calgary.

Let's start by importing the necessary libraries:

In [1]:
# import necessary libraries
import meshflow # version v0.1.0-dev1

import os # python 3.10.2

Now, let's provide necessary information to set up the `MESH` model:

In [11]:
# main work path - modify
work_path = os.path.join(os.getcwd(), '..')

# using meshflow==v0.1.0-dev1
# modify each segment to match your settings
config = {
    'riv': os.path.join(work_path, '0-geofabric', 'rivers.shp'),
    'cat': os.path.join(work_path, '0-geofabric', 'basins.shp'),
    'landcover': os.path.join(work_path, 'gistool-outputs', 'gis_stats_NA_NALCMS_landcover_2015v3_30m.csv'),
    'forcing_files': os.path.join(work_path, 'easymore-outputs'),
    'forcing_vars': [ # does the variable list, match those of the "agnostic" step?
        "RDRS_v2.1_P_P0_SFC",
        "RDRS_v2.1_P_HU_09944",
        "RDRS_v2.1_P_TT_09944",
        "RDRS_v2.1_P_UVC_09944",
        "RDRS_v2.1_A_PR0_SFC",
        "RDRS_v2.1_P_FB_SFC",
        "RDRS_v2.1_P_FI_SFC",
    ],
    'forcing_units': { # Here, enter RDRS's original variable units
        'RDRS_v2.1_P_P0_SFC': 'millibar',
        'RDRS_v2.1_P_HU_09944': 'kg/kg',
        'RDRS_v2.1_P_TT_09944': 'celsius',
        'RDRS_v2.1_P_UVC_09944': 'knot',
        'RDRS_v2.1_A_PR0_SFC': 'm/hr',
        'RDRS_v2.1_P_FB_SFC': 'W/m^2',
        'RDRS_v2.1_P_FI_SFC': 'W/m^2',
    },
    'forcing_to_units': { # And here, the units that MESH needs to read
         'RDRS_v2.1_P_UVC_09944': 'm/s',
         'RDRS_v2.1_P_FI_SFC': 'W/m^2',
         'RDRS_v2.1_P_FB_SFC': 'W/m^2',
         'RDRS_v2.1_A_PR0_SFC': 'mm/s',
         'RDRS_v2.1_P_P0_SFC': 'pascal',
         'RDRS_v2.1_P_TT_09944': 'kelvin',
         'RDRS_v2.1_P_HU_09944': 'kg/kg',
    },
    'main_id': 'DrainID', # what is the main ID of each river segment? Column name in the `cat` Shapefile
    'ds_main_id': 'NextDrain', # what is the downstream segment ID for each river segment? ditto.
    'landcover_classes': { # these are the classes defined for NALCMS-Landsat 2015 dataset. Is this accurate?
        0: 'Unknown',
        1: 'Temperate or sub-polar needleleaf forest',
        2: 'Sub-polar taiga needleleaf forest',
        3: 'Tropical or sub-tropical broadleaf evergreen forest',
        4: 'Tropical or sub-tropical broadleaf deciduous forest',
        5: 'Temperate or sub-polar broadleaf deciduous forest',
        6: 'Mixed forest',
        7: 'Tropical or sub-tropical shrubland',
        8: 'Temperate or sub-polar shrubland',
        9: 'Tropical or sub-tropical grassland',
        10: 'Temperate or sub-polar grassland',
        11: 'Sub-polar or polar shrubland-lichen-moss',
        12: 'Sub-polar or polar grassland-lichen-moss',
        13: 'Sub-polar or polar barren-lichen-moss',
        14: 'Wetland',
        15: 'Cropland',
        16: 'Barren lands',
        17: 'Urban',
        18: 'Water',
        19: 'Snow and Ice',
    },
    'ddb_vars': { # the stuff that MESH needs: slope, river length, etc... Let me know if there is any issues here!
        'Slope': 'ChnlSlope',
        'Shape_Leng': 'ChnlLength',
        'Rank': 'Rank',
        'NextDrain': 'Next',
        'landcover': 'GRU',
        'Shape_Area': 'GridArea',
        'landcover_names': 'LandUse',
    },
    'ddb_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'km', # is it in km or m? Please check the units of the Shapefile you created!
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'km^2', # what was the unit of the GridArea, or Shape_Area in the `catchments` Shapefile?
        'LandUse': 'dimensionless',
    },
    'ddb_to_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'm', # This is what MESH needs, no need to change.
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'm^2', # This is what MESH needs, no need to change.
        'LandUse': 'dimensionless',
    },
    'ddb_min_values': {
        'ChnlSlope': 1e-10, # in case there are 0s in the `rivers` Shapefile, we need minimums for certain variables
        'ChnlLength': 1e-3,
        'GridArea': 1e-3,
    },
    'gru_dim': 'NGRU', # change to `NGRU` for 'MESH>=r1860', keep for 'MESH<=1860', for example for r1813.
    'hru_dim': 'subbasin',
    'outlet_value': -9999,
}

We can build an "instance" of the workflow class:

In [12]:
exp1 = meshflow.MESHWorkflow(**config)

And, we can run it using:

In [13]:
exp1.run()

ValueError: conflicting sizes for dimension 'NGRU': length 1 on 'landcover_names' and length 0 on {'NGRU': 'NGRU'}

Once the run is finished, we can checkout the forcing and drainage database file:

In [11]:
exp1.forcing

<xarray.Dataset> Size: 6kB
Dimensions:                (subbasin: 0, time: 744)
Coordinates:
  * time                   (time) datetime64[ns] 6kB 2018-01-01T13:00:00 ... ...
  * subbasin               (subbasin) int16 0B 
Data variables:
    RDRS_v2.1_P_P0_SFC     (subbasin, time) float64 0B 
    RDRS_v2.1_P_HU_09944   (subbasin, time) float64 0B 
    RDRS_v2.1_P_TT_09944   (subbasin, time) float64 0B 
    RDRS_v2.1_P_UVC_09944  (subbasin, time) float64 0B 
    RDRS_v2.1_A_PR0_SFC    (subbasin, time) float64 0B 
    RDRS_v2.1_P_FB_SFC     (subbasin, time) float64 0B 
    RDRS_v2.1_P_FI_SFC     (subbasin, time) float64 0B 
    lat                    (subbasin) float64 0B 
    lon                    (subbasin) float64 0B 
    crs                    int64 8B 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create forcing .nc file for MESH
    Conventions:  CF-1.6
    history:      Created using MESHFlow version v0.1.0.dev2

In [13]:
exp1.ddb

<xarray.Dataset> Size: 3kB
Dimensions:     (subbasin: 0, NGRU: 13)
Coordinates:
  * subbasin    (subbasin) int16 0B 
Dimensions without coordinates: NGRU
Data variables:
    ChnlSlope   (subbasin) float64 0B 
    ChnlLength  (subbasin) float64 0B 
    Rank        (subbasin) float64 0B 
    Next        (subbasin) float64 0B 
    GRU         (subbasin, NGRU) float64 0B 
    GridArea    (subbasin) float64 0B 
    LandUse     (NGRU) <U49 3kB 'Temperate or sub-polar needleleaf forest' .....
    lat         (subbasin) float64 0B 
    lon         (subbasin) float64 0B 
    crs         int64 8B 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create a drainage database .nc file for MESH
    featureType:  point

In [7]:
# create a directory for MESH setup
try:
    os.makedirs(os.path.join(work_path, '4-outputs', 'mesh-bow-at-calgary'))
except FileExistsError:
    pass

# saving drainage database and forcing files
exp1.save(os.path.join(work_path, '4-outputs', 'mesh-bow-at-calgary'))

/home/kasra.keshavarz1/virtual-envs/test-env/lib/python3.11/site-packages/meshflow/core.py:401: UserWarning: MESH settings need manual adjustments. Automation is a work in progress...
  warnings.warn(


____

If there is any issue, open a ticket on [MESHFlow's GitHub Webpage](https://github.com/kasra-keshavarz/meshflow/issues).